---

## Problem Definition

Predict the performance of companies according to **`rating`** and **`work happiness score`**
- Analyze the `importance` of features help in increasing work happiness score.
- Find the factors affecting ratings across different `industries` to provide comprehensive improvement strategies
- Detect the `anomalies` of ratings.

---

### Split Dataset into Train & Test
Split the whole dataset into **`train`** and **`test`** datasets with the ratio 8 : 2.


In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
df = pd.read_pickle('../dataset/data_prepare2.pickle')
train, test = train_test_split(df, test_size = 0.2, random_state = 42, shuffle = False)
train.to_excel('train.xlsx', index = False)
test.to_excel('test.xlsx', index = False)